In [1]:
import tensorflow as tf
tf.debugging.set_log_device_placement(True)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch==1.5.1 torchvision==0.6.1
!pip install pyTelegramBotAPI==3.8.0
!pip install nvgpu
!pip install ISR
!pip install 'h5py==2.10.0' --force-reinstall

     |████████████████████████████████| 753.2 MB 14 kB/s 
     |████████████████████████████████| 6.6 MB 22.3 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.10.0+cu102
    Uninstalling torchvision-0.10.0+cu102:
      Successfully uninstalled torchvision-0.10.0+cu102
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.5.1 which is incompatible.
     |████████████████████████████████| 98 kB 4.3 MB/s 
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-3.8.0-py3-none-any.whl size=75053 sha256=5f29639cb389c37c3431639ad1ed35b4b0621a0f692551eef4d40cfb4d53b90d
  Stored in directory: /root/.cache/

     |████████████████████████████████| 2.9 MB 7.5 MB/s 
     |████████████████████████████████| 15.7 MB 49.6 MB/s 
  Attempting uninstall: six
    Found existing installation: six 1.15.0
    Uninstalling six-1.15.0:
      Successfully uninstalled six-1.15.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.5.1 which is incompatible.
tensorflow-probability 0.13.0 requires gast>=0.3.2, but you have gast 0.2.2 which is incompatible.
google-colab 1.0.0 requires six~=1.15.0, but you have six 1.16.0 which is incompatib

In [ ]:
cd /content/drive/MyDrive/MFTI/Project_pro

/content/drive/MyDrive/MFTI/Project_pro


In [ ]:
import requests
import json
import time
from PIL import Image
import cv2
from io import BytesIO
from ISR.models import RDN
import nvgpu
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torchvision.utils import save_image
rdn = RDN(weights='noise-cancel')
import telebot
from predictor2 import Predictor
from predictor3 import Predictor1
from predictor4 import Predictor2
from predictor5 import Predictor3
from telebot import types
import random



API_KEY = '1959509632:AAEjJ58I52ncXre0BkpI5_80zQKhi6tVFpo'
USER_DATA = {}


print('Start!')

def imshow(image, title=None):
    plt.imshow(image / np.max(image))
    if title is not None:
        plt.title(title)
    plt.show()
        
bot = telebot.TeleBot(API_KEY,threaded=True)
predictors = [rdn, Predictor(), Predictor1(), Predictor3(), Predictor2()]
test = 0
def downloadFile(filepath):
    file = requests.get('https://api.telegram.org/file/bot{}/{}'\
                       .format(API_KEY,filepath))
    img = Image.open(BytesIO(file.content))
    return np.array(img)


@bot.message_handler(commands=['start'])
def start_message(message):
    text = "Привет! Я Style Transfer Bot!\nЯ могу преобразовать фото в стиле другого фото!"
    if not(message.chat.id in USER_DATA):
        USER_DATA[message.chat.id] = {}
    USER_DATA[message.chat.id]['iscontent'] = True
    USER_DATA[message.chat.id]['isstyle'] = False
    bot.send_message(message.chat.id, text)
    bot.send_message(message.chat.id, "1) Загрузи фото 🏞, из которого будет извлечен контент")
@bot.message_handler(commands=['help'])
def help_message(message):
    text = "Напиши /start и следуй инструкциям."
    bot.send_message(message.chat.id, text)

        
@bot.message_handler(content_types=['photo']) 
def photo_message(message): #В этой функции юот получает контент и стиль
    chat_id = message.chat.id
    if not(message.chat.id in USER_DATA):
        USER_DATA[message.chat.id] = {} 
    if ('inprocess' in USER_DATA[chat_id]) and (USER_DATA[chat_id]['inprocess']):
        bot.send_message(chat_id, 'Сначала подождите, пока обработается предыдущее фото!')
        return None
    if not('iscontent' in USER_DATA[chat_id]) or USER_DATA[chat_id]['iscontent'] or \
       (not(USER_DATA[chat_id]['iscontent']) and not(USER_DATA[chat_id]['isstyle']))\
        and not(USER_DATA[chat_id]['ispredictor']):

        USER_DATA[chat_id]['content'] = bot.get_file(message.photo[-1].file_id).file_path
        USER_DATA[chat_id]['iscontent'] = False
        USER_DATA[chat_id]['isstyle'] = True
        USER_DATA[chat_id]['ispredictor'] = False
        bot.send_message(chat_id,'2) Загрузи фото 🎆, из которого будет извлечен стиль')
    elif USER_DATA[chat_id]['isstyle']:

        USER_DATA[chat_id]['style'] = bot.get_file(message.photo[-1].file_id).file_path
        USER_DATA[chat_id]['isstyle'] = False
        USER_DATA[chat_id]['ispredictor'] = True
    
        bot.send_message(chat_id, '''Теперь тебе необходимо выбрать модель. \n\
Сейчас я предоставлю тебе на выбор 4 модели: \n\
    1. Cтабильная версия переноса стиля.\n\
    2. Cтиль перенесётся круто, но это не точно.\n\
    3. Стабильная версия переноса стиля, где все цвета стиля. (рекомендую) \n
    4. Cтиль перенесётся круто, но это не точно c переносом цвета стиля.\n\
Если результат не понравится, то всегда  можно выбрать другую модель :)''')
        
        markup = types.ReplyKeyboardMarkup(True, True)
        itembtn1 = types.KeyboardButton('1')
        itembtn2 = types.KeyboardButton('2')
        itembtn3 = types.KeyboardButton('3')
        itembtn4 = types.KeyboardButton('4')

        markup.row(itembtn1, itembtn2)
        markup.row(itembtn3, itembtn4)

        bot.send_message(chat_id, 'Теперь просто нажми на кнопку или отправь цифру от 1 до 4.', reply_markup=markup)
        USER_DATA[chat_id]['stop'] = False
        
@bot.message_handler(commands=['stop'])  
def stop(message):
    chat_id = message.chat.id
    if not(message.chat.id in USER_DATA):
        USER_DATA[message.chat.id] = {}
        
    if 'pred' in USER_DATA[chat_id]:
        USER_DATA[chat_id]['stop'] = True
        USER_DATA[chat_id]['pred'].stop_predictor()        
        
        
    else:
        bot.send_message(chat_id, 'Чтобы остановить процесс, его нужно сначала запустить')
        
@bot.message_handler(content_types=['text'])
def get_predictor(message): a
    chat_id = message.chat.id
    mode = {1:'basic', 2:'match', 3:'match_style'} 
    if not(message.chat.id in USER_DATA):
        USER_DATA[message.chat.id] = {} 
    if ('content' in USER_DATA[chat_id]) and ('style' in USER_DATA[chat_id]) and (USER_DATA[chat_id]['ispredictor']) and ('ispredictor' in USER_DATA[chat_id]):
        if not(type(message.text) == 'NonType'):
            if str(message.text).isnumeric():
                if 1 <= int(message.text) <= 5:
                    print('yes')
                    USER_DATA[chat_id]['num_pred'] = int(message.text)
                    USER_DATA[chat_id]['pred'] = predictors[int(message.text)]
                    USER_DATA[chat_id]['ispredictor'] = False
                    USER_DATA[chat_id]['inprocess'] = False
                    
                    
                        
                else:
                    bot.send_message(chat_id, 'Я же говорил,что цифру от 1 до 4 🙄')
            else:
                bot.send_message(chat_id, 'Цифру, а не текст 😡')
       
                    
    if 'content' in USER_DATA[chat_id] and 'style' in USER_DATA[chat_id] and 'pred' in USER_DATA[chat_id]:
        
        markup = types.ReplyKeyboardRemove(selective=False)
        bot.send_message(chat_id, 'Начинаю обработку...', reply_markup=markup)
        
        if not(USER_DATA[chat_id]['num_pred'] == 5):
            bot.send_message(chat_id, 'В течение 10-15 минут будет предоставлено несколько вариантов изображнения.')
        else:
            bot.send_message(chat_id, 'Так как была выбрана модель без сжатия фотографии, то обработка займёт около 30 минут.')
        
            
        USER_DATA[chat_id]['inprocess'] = True
        
        plt.figure(figsize=[8,4])
        plt.subplot(1,2,1)
        imshow(downloadFile(USER_DATA[chat_id]['style']).data, title='Style Image')
        plt.subplot(1,2,2)
        imshow(downloadFile(USER_DATA[chat_id]['content']).data, title='Content Image')
        
        
        im_genr = USER_DATA[chat_id]['pred'].get_image_predict(downloadFile(USER_DATA[chat_id]['style']),
                                         downloadFile(USER_DATA[chat_id]['content']), 
                                         mode='match_style')
        for tens in im_genr: #обработанные фото выходят из предиктора
            image = tens.clone().cpu()
            image = image.view(tens.shape[1:4])  
            image = image.detach().numpy().transpose([1,2,0])*255
            if not(USER_DATA[chat_id]['num_pred'] == 5):
                image = rdn.predict(image)
            dst = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            success, encoded_image = cv2.imencode('.jpg', dst)
            content = encoded_image.tobytes()
            bot.send_photo(chat_id,content)
            if USER_DATA[chat_id]['stop']:
                break
            
        bot.send_message(chat_id, 'Обработка завершена.')
        bot.send_message(chat_id, 'Спасибо за ожидание! Теперь ты можешь выбрать понравившиеся фотографии и использовать их как и где захочется.')
        bot.send_message(chat_id, 'Теперь, чтобы запустить меня еще раз, то отправь фото контента 🏞, или просто напиши /start')
        
        USER_DATA[chat_id]['inprocess'] = False
        print(USER_DATA)
        
        
        
if __name__ == '__main__':
    bot.polling()

IndentationError: ignored

In [ ]:
   if USER_DATA[chat_id]['num_pred'] == 3 or USER_DATA[chat_id]['num_pred'] == 4 and  'pred' in USER_DATA[chat_id]:
            USER_DATA[chat_id]['ismode'] = True
            markup = types.ReplyKeyboardRemove(selective=False)
            bot.send_message(chat_id, '''Так как ты выбрал модель с переносом цвета, то необходимо выбрать режим: \n\
Предоставляю тебе шанс выбрать мод: \n\
    1. Стандартный перенос стиля. \n\
    2. Сохранить цвета контента и перенести только форму стиля. \n\
    3. Перенести ещё и цвет стиля.''', reply_markup=markup)
            markup = types.ReplyKeyboardMarkup()
            itembtn1 = types.KeyboardButton('1')
            itembtn2 = types.KeyboardButton('2')
            itembtn3 = types.KeyboardButton('3')
            markup.row(itembtn1, itembtn2, itembtn3)
            bot.send_message(chat_id, 'Теперь просто нажми на кнопку или отправь цифру от 1 до 3', reply_markup=markup)
            if USER_DATA[chat_id]['ismode']:
                if not(type(message.text) == 'NonType'):
                    if str(message.text).isnumeric():
                        print('yes')
                        if 1 <= int(message.text) <= 3:
                            USER_DATA[chat_id]['mode'] = mode[int(message.text)]
                        else:
                            bot.send_message(chat_id, 'Я же говорил,что цифру от 1 до 3 🙄 \n\
    Ты вообще читаешь, что я тебе пишу?')
                    else:
                        bot.send_message(chat_id, 'Цифру, а не текст 😡')     
        else:
            USER_DATA[chat_id]['ismode'] = False
            USER_DATA[chat_id]['mode'] = mode[1] 